# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests as req
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
wdf = pd.read_csv("../WeatherPy/output_data/cities.csv")
wdf

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,Bredasdorp,ZA,-34.53,20.04,44.60,87,1.12,99,1597645565
1,Cape Town,ZA,-33.93,18.42,48.99,81,9.17,20,1597645323
2,Camacha,PT,33.08,-16.33,71.60,83,6.93,20,1597645566
3,Atuona,PF,-9.80,-139.03,77.97,77,16.51,2,1597645566
4,Tupik,RU,54.43,119.93,60.48,52,12.08,100,1597645566
...,...,...,...,...,...,...,...,...,...
568,Alugan,PH,12.22,125.48,86.09,70,4.41,99,1597645709
569,Iqaluit,CA,63.75,-68.51,41.00,93,3.36,40,1597645709
570,Artesia,US,33.87,-118.08,73.40,83,3.36,90,1597645709
571,Oriximiná,BR,-1.77,-55.87,78.04,68,4.14,50,1597645710


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Setting gmaps key so we can call the API
gmaps.configure(api_key = g_key)

# gmaps heatmap weights need to be floats, so we convert the humidity
# column to floats to avoid problems later
wdf = wdf.astype({"Humidity": float})

# Getting location coordinates and humidity data
locations = wdf[["Lat", "Lng"]]
humidity = wdf["Humidity"]

In [6]:
fig = gmaps.figure()

# Create and add a heat layer to the gmaps figure
hum_heat = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=np.max(humidity), point_radius=4)
fig.add_layer(hum_heat)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Conditions:
# - A max temperature lower than 80 degrees but higher than 70 (Assuming inclusive conditionals)
# - Wind speed less than 10 mph (Assuming inclusive conditional)
# - Zero cloudiness

condition_cities = wdf.loc[(wdf["Max Temp"] >= 70) & \
                           (wdf["Max Temp"] <= 80) & \
                           (wdf["Wind Speed"] <= 10) & \
                           (wdf["Cloudiness"] == 0)].dropna()

condition_cities

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Date
83,Mocuba,MZ,-16.84,36.99,74.03,70.0,4.27,0,1597645589
118,Ampanihy,MG,-24.70,44.75,71.02,66.0,6.35,0,1597645598
207,Odesa,UA,46.48,30.73,75.20,44.0,6.02,0,1597645485
244,San Quintín,MX,30.48,-115.95,74.01,71.0,5.46,0,1597645629
271,Hervey Bay,AU,-25.30,152.85,73.40,27.0,9.17,0,1597645636
437,Machinga,MW,-15.17,35.30,73.38,40.0,5.21,0,1597645677
442,Province of Siena,IT,43.22,11.40,75.00,64.0,0.87,0,1597645678
454,Beloha,MG,-25.17,45.05,72.45,65.0,5.08,0,1597645681
497,Novovasylivka,UA,46.83,35.75,73.83,36.0,5.35,0,1597645692
506,Genç,TR,38.75,40.56,78.80,36.0,6.93,0,1597645694


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Keep only the data we need for the gmaps API calls and adding Hotel Name
# column to save results from the API calls
hotel_df = condition_cities[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

# Suppress warning for chained assignments for raising false positives
pd.options.mode.chained_assignment = None

hotel_df

,City,Country,Lat,Lng,Hotel Name
83,Mocuba,MZ,-16.84,36.99,
118,Ampanihy,MG,-24.70,44.75,
207,Odesa,UA,46.48,30.73,
244,San Quintín,MX,30.48,-115.95,
271,Hervey Bay,AU,-25.30,152.85,
437,Machinga,MW,-15.17,35.30,
442,Province of Siena,IT,43.22,11.40,
454,Beloha,MG,-25.17,45.05,
497,Novovasylivka,UA,46.83,35.75,
506,Genç,TR,38.75,40.56,


In [10]:
# Setting up API request URL and template for params needed to request
# information from gmaps
api_call = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"key" : g_key,
         "type" : "hotel",
         "keyword" : "hotel",
         "radius": 5000,
         "location": ""}

In [16]:
# Loop to iterate through every row in the dataframe
for i, row in hotel_df.iterrows():
    
    # We get latitude and longitude for the API Call param "location"
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add location coordinates to params for API call
    params["location"] = f"{latitude},{longitude}"
    
    # Get response from API call in JSON format
    resp = req.get(api_call, params = params).json()
    
    # We get the results section from the JSON
    r = resp["results"]
    
    # If the results section is not empty, we add it to the row's Hotel Name
    # column, if not, we print the "Hotel not found" message
    if r:
        hotel_df.loc[i, "Hotel Name"] = r[0]["name"]
    else:
        print("No hotel was found in a 5 Km radius from", 
              latitude, 
              longitude, 
              "located in", 
              row["City"], 
              row["Country"])
    
hotel_df

No hotel was found in a 5 Km radius from -24.7 44.75 located in Ampanihy MG
No hotel was found in a 5 Km radius from -15.17 35.3 located in Machinga MW
No hotel was found in a 5 Km radius from -25.17 45.05 located in Beloha MG
No hotel was found in a 5 Km radius from 46.83 35.75 located in Novovasylivka UA
No hotel was found in a 5 Km radius from 38.75 40.56 located in Genç TR


,City,Country,Lat,Lng,Hotel Name
83,Mocuba,MZ,-16.84,36.99,Pensão Venus
118,Ampanihy,MG,-24.70,44.75,
207,Odesa,UA,46.48,30.73,Panorama De Luxe
244,San Quintín,MX,30.48,-115.95,Hotel Jardines Baja
271,Hervey Bay,AU,-25.30,152.85,Oaks Hervey Bay Resort and Spa
437,Machinga,MW,-15.17,35.30,
442,Province of Siena,IT,43.22,11.40,Borgo Antico hotel e ristorante
454,Beloha,MG,-25.17,45.05,
497,Novovasylivka,UA,46.83,35.75,
506,Genç,TR,38.75,40.56,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer on top of heat map
mrkrs = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(mrkrs)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))